In [11]:
import numpy as np
import pandas as pd

In [12]:
%%bash
ls -lh ../data/raw

total 3.4M
-rw-r--r-- 1 root root  159 Jul 12 11:43 prompts_test.csv
-rw-r--r-- 1 root root  16K Jul 12 11:43 prompts_train.csv
-rw-r--r-- 1 root root  111 Jul 12 11:43 sample_submission.csv
-rw-r--r-- 1 root root  166 Jul 12 11:43 summaries_test.csv
-rw-r--r-- 1 root root 3.3M Jul 12 11:43 summaries_train.csv


In [15]:
prompts_train = pd.read_csv('../data/raw/prompts_train.csv')
prompts_test = pd.read_csv('../data/raw/prompts_test.csv')
sample_submission = pd.read_csv('../data/raw/sample_submission.csv')
summaries_train = pd.read_csv('../data/raw/summaries_train.csv')
summaries_test = pd.read_csv('../data/raw/summaries_test.csv')

In [ ]:
a = np.arange(10)

In [ ]:
a